# <font size="65">Benchmarking Robust Classification Under Data Imbalance</font>

Trabalho realizado por:

* Afonso Coelho (FCUP_IACD:202305085)  
* Diogo Amaral (FCUP_IACD:202305187)  
* Miguel Carvalho (FCUP_IACD:202305229)  

<div style="padding: 10px;padding-left:5%">
<img src="fotos_md/Cienciasporto.png" style="float:left; height:75px;width:200px">
<img src="fotos_md/Feuporto.png" style="float:left ; height:75px; padding-left:20px;width:200px">
</div>

<div style="clear:both;"></div>

******

## 0. Projeto

Este projeto insere-se na unidade curricular de *Machine Learning I (CC2008)* e tem como objetivo aprofundar a compreensão teórica e prática de algoritmos de classificação supervisionada, aplicando-os em cenários com características desafiantes nos dados.  

Em particular, a nossa abordagem foca-se na **classificação binária com dados desbalanceados**, uma situação comum em domínios médicos, financeiros e industriais, onde uma das classes é significativamente mais rara do que a outra.  

Partimos de uma implementação base de um algoritmo clássico de classificação, que modificamos para lidar melhor com este tipo de desbalanceamento. O desempenho do modelo original e do modelo modificado será comparado em vários conjuntos de dados de benchmark, com o objetivo de validar empiricamente a eficácia das alterações propostas.  

O projeto decorre em duas fases:
- **Fase 1**: Análise e avaliação da versão original do algoritmo.
- **Fase 2**: Proposta e teste de uma versão modificada, mais robusta ao desbalanceamento.

Todos os algoritmos são implementados **de raiz**, sem recurso a bibliotecas de alto nível como `scikit-learn`, respeitando os requisitos do enunciado.




# Imports
Importante correr com env feito pelo ficheiro [requirements](requirements.txt).

In [ ]:
import logging

from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from mla.metrics.metrics import mean_squared_error
from mla.neuralnet import NeuralNet
from mla.neuralnet.constraints import MaxNorm
from mla.neuralnet.layers import Activation, Dense, Dropout
from mla.neuralnet.optimizers import Adadelta, Adam,RMSprop
from mla.neuralnet.parameters import Parameters
from mla.neuralnet.regularizers import L2
from mla.utils import one_hot
import pandas as pd
import os
import numpy as np
import importlib
import json


In [ ]:
def setupx_y(dataset_path):
    data = pd.read_csv(dataset_path)
    
    # Supondo que o dataset tenha colunas 'features' e 'target'
    X = data.iloc[:, :-1].values  # Todas as colunas menos a última como features
    y = data.iloc[:, -1].values   # Última coluna como target

    values, counts = np.unique(y, return_counts=True)
    majority_ratio = counts.max() / counts.sum()

    if len(values) != 2:
        raise ValueError("A coluna target não é binária")

    if majority_ratio <= 0.6:
        raise ValueError("Nenhuma classe é majoritária (>60%)")

    majority_class = values[np.argmax(counts)]
    minority_class = values[np.argmin(counts)]

    # Re-encode: majority -> 0, minority -> 1
    y = np.where(y == majority_class, 0, 1)

    # Count the occurrences of 0's and 1's in the target column
    count_zeros = np.sum(y == 0)
    count_ones = np.sum(y == 1)
    print(f"Count of 0's: {count_zeros}, Count of 1's: {count_ones}")


    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    
    y = one_hot(y)
    return X, y, count_zeros, count_ones
    

import math

def calcular_batch_size(n_total, n_minor, k=3, batch_min=32, batch_max=512):
    """
    Calcula um batch size adequado com base no desequilíbrio entre classes.
    
    Parâmetros:
    - n_total: total de amostras
    - n_minor: número de amostras da classe minoritária
    - k: fator multiplicador de segurança (padrão = 3)
    - batch_min: tamanho mínimo aceitável (padrão = 32)
    - batch_max: tamanho máximo permitido (padrão = 512)

    Retorna:
    - batch_size ajustado
    """
    if n_minor == 0:
        raise ValueError("Número de amostras minoritárias não pode ser zero.")

    imbalance_ratio = (n_total - n_minor) / n_minor
    estimated_batch = math.ceil(k * imbalance_ratio)

    # Respeita os limites definidos
    batch_size = max(batch_min, estimated_batch)
    batch_size = min(batch_size, n_total, batch_max)

    return batch_size

def custom_stratified_split(X, y_onehot, test_size=0.15, minority_ratio_test=0.10, random_state=1111):
    from sklearn.model_selection import train_test_split
    import numpy as np

    # Converter one-hot para binário
    y = np.argmax(y_onehot, axis=1)

    # Separar classes
    unique, counts = np.unique(y, return_counts=True)
    if len(unique) != 2:
        raise ValueError("Target y deve ser binário")
    minority_class = unique[np.argmin(counts)]
    majority_class = unique[np.argmax(counts)]

    # Separar X e y
    X_min, y_min = X[y == minority_class], y[y == minority_class]
    X_maj, y_maj = X[y == majority_class], y[y == majority_class]

    # Split da classe minoritária: 90% treino, 10% teste
    X_min_train, X_min_test, y_min_train, y_min_test = train_test_split(
        X_min, y_min, test_size=minority_ratio_test, stratify=y_min, random_state=random_state
    )

    # Calcular total de amostras para o conjunto de teste
    total_test_size = int(test_size * len(y))
    n_remaining = total_test_size - len(y_min_test)
    if n_remaining < 0:
        raise ValueError("minority_ratio_test demasiado alto para o tamanho total de teste")

    # Split da classe majoritária para preencher o resto do teste
    X_maj_train, X_maj_test, y_maj_train, y_maj_test = train_test_split(
        X_maj, y_maj, test_size=n_remaining, stratify=y_maj, random_state=random_state
    )

    # Combinar splits
    X_train = np.concatenate([X_min_train, X_maj_train])
    y_train = np.concatenate([y_min_train, y_maj_train])
    X_test = np.concatenate([X_min_test, X_maj_test])
    y_test = np.concatenate([y_min_test, y_maj_test])

    from sklearn.utils import shuffle
    X_train, y_train = shuffle(X_train, y_train, random_state=random_state)
    X_test, y_test = shuffle(X_test, y_test, random_state=random_state)

    # Re-transformar para one-hot
    y_train_onehot = np.eye(2)[y_train]
    y_test_onehot = np.eye(2)[y_test]

    return X_train, X_test, y_train_onehot, y_test_onehot
def classification(dataset_path,antes,filename):

    # Carregar o dataset
    X, y, count_zeros, count_ones = setupx_y(dataset_path)
    
    X_train, X_test, y_train, y_test = custom_stratified_split(X, y, test_size=0.15, minority_ratio_test=0.10,random_state=1111)
    arr = np.asarray(y_train)

    if arr.ndim > 1 and arr.shape[1] > 1:
        # one-hot → labels 0/1
        labels = np.argmax(arr, axis=1)
    else:
        labels = arr.flatten()

    zeros = np.sum(labels == 0)
    uns   = np.sum(labels == 1)

    print(f"Nº de zeros: {zeros}")
    print(f"Nº de uns  : {uns}")

    batchsize= calcular_batch_size(X_train.shape[0],np.sum(y_train == 1), k=3, batch_min=8, batch_max=X_train.shape[0])
    if antes==0:
        model = NeuralNet(
            layers=[
                Dense(128, Parameters(init="uniform", regularizers={"W": L2(1e-3)})),
                Activation("relu"),
                Dropout(0.5),
                Dense(64, Parameters(init="uniform", regularizers={"W": L2(1e-3)})),
                Activation("relu"),
                Dropout(0.3),
                Dense(2),
                Activation("sigmoid"),
            ],
            filename=filename,
            loss="focal_loss",
            testarerros=True,
            zeros=zeros,
            uns=uns,
            count_ones=count_ones,
            count_zeros=count_zeros,
            optimizer=RMSprop(),
            metric="f1score",
            batch_size=batchsize,
            equilibrar_batches=True,
            max_epochs=30,
            shuffle=True,
            l2=True,
            dropout=True
        )
    elif antes==1:
        model = NeuralNet(
            layers=[
                Dense(128),
                Activation("relu"),
                Dense(64),
                Activation("relu"),
                Dense(2),
                Activation("sigmoid"),
            ],
            filename=filename,
            loss="binary_crossentropy",
            testarerros=True,
            zeros=zeros,
            uns=uns,
            count_ones=count_ones,
            count_zeros=count_zeros,
            optimizer=RMSprop(),
            metric="f1score",
            equilibrar_batches=True,
            max_epochs=30,
            shuffle=True,
            l2=False,
            dropout=False
        )
    elif antes==2:
        model = NeuralNet(
            layers=[
                Dense(128, Parameters(init="uniform", regularizers={"W": L2(1e-3)})),
                Activation("relu"),
                Dropout(0.5),
                Dense(64, Parameters(init="uniform", regularizers={"W": L2(1e-3)})),
                Activation("relu"),
                Dropout(0.3),
                Dense(2),
                Activation("sigmoid"),
            ],
            filename=filename,
            loss="automatic_weighted_binary_crossentropy",
            testarerros=True,
            zeros=zeros,
            uns=uns,
            count_ones=count_ones,
            count_zeros=count_zeros,
            optimizer=RMSprop(),
            metric="f1score",
            batch_size=batchsize,
            equilibrar_batches=True,
            max_epochs=30,
            shuffle=True,
            l2=True,
            dropout=True
        )
    model.fit(X_train, y_train,X_test,y_test)
    return model



# Criar modelos e treinar os modelos

> Nota: Demora cerca de 15 min 

In [ ]:
cwd = os.getcwd()  
dataset_dir = os.path.join(cwd, "fixed_datasets") 
modelos=[]
for filename in os.listdir(dataset_dir):
    if filename.endswith(".csv"):
        dataset_path = os.path.join(dataset_dir, filename)
        for i in range(3):
            modelos.append(classification(dataset_path,i,filename))

In [ ]:
def _sanitize_data(obj):
    """
    Recursively convert numpy objects to native Python types for JSON serialization.
    """
    if isinstance(obj, np.ndarray):
        return _sanitize_data(obj.tolist())
    elif isinstance(obj, (list, tuple)):
        return [_sanitize_data(item) for item in obj]
    elif isinstance(obj, np.generic):  # numpy scalar
        return obj.item()
    else:
        return obj

def adicionar_modelo_ao_dataset(model, dataset=None):
    """
    Adiciona os dados de um modelo ao dataset, convertendo one-hot para labels binárias.
    """
    # Converter one-hot para labels binárias (0 ou 1)
    y_train = getattr(model, 'y', None)
    y_test = getattr(model, 'Y_test', None)
    
    # Se os dados estão em one-hot, extrair as labels
    if y_train is not None and len(y_train.shape) > 1:
        y_train = np.argmax(y_train, axis=1)  # Assume one-hot na última dimensão
    if y_test is not None and len(y_test.shape) > 1:
        y_test = np.argmax(y_test, axis=1)
    
    # Preparar dados sanitizados
    nova_linha = {
        'ficheiro': model.filename,
        'epocas': int(getattr(model, 'max_epochs', 0)),
        'loss_nome': getattr(model, 'loss_name', None),
        'l2': float(getattr(model, 'l2', 0.0)),
        'dropout': float(getattr(model, 'dropout', 0.0)),
        'zeros': int(getattr(model, 'zeros', 0)),
        'uns': int(getattr(model, 'uns', 0)),
        'train_out': json.dumps(_sanitize_data(y_train)),  # Labels binárias
        'test_out': json.dumps(_sanitize_data(y_test)),    # Labels binárias
        'probs': json.dumps(_sanitize_data(getattr(model, 'metric_list', None))),
        'loss_list': json.dumps(_sanitize_data(getattr(model, 'loss_list', None))),
    }

    # Criar ou atualizar o DataFrame
    linha_df = pd.DataFrame([nova_linha])
    if dataset is None:
        dataset = linha_df
    else:
        dataset = pd.concat([dataset, linha_df], ignore_index=True)
    
    return dataset

# Gerar o dataset com os modelos Aprox 350MB

> Nota: Demora cerca de 30sec

In [ ]:
csv_file = "resultados.csv"

for i, model in enumerate(modelos):
    linha_df = adicionar_modelo_ao_dataset(model)
    if i == 0:
        # Cria o arquivo com cabeçalho
        linha_df.to_csv(csv_file, index=False, mode='w', header=True)
    else:
        # Anexa sem cabeçalho
        linha_df.to_csv(csv_file, index=False, mode='a', header=False)

# Plotar métricas 
**processar_modelos_subset**: Todas as métricas com True de cada ficheiro. Modo visual pode ser:
* Cada - um gráfico para cada loss e métrica.
* Juntos - um gráfico com todas as losses num so gráfico.
* Todos - Os gráficos de cada e Todos ``` Atenção gera +500 gráficos não é aconselhado para visualização```.

**plotar_gráficos**: Um gráfico do racio das classes * nr de amostras pelas métricas com True  de todos os ficheiros num so gráfico. Mode visual pode ser:
* Cada - um gráfico para cada loss e metrica.
* Juntos - um gráfico com todas as losses num so gráfico.
* Todos - Os gráficos de cada e Todos.

**gerar_tabelas_percentuais_métricas**: Gerar 3 tabelas com cada metrica e o nr de amostras que surpassam a percentagem da coluna.

In [ ]:
from fazerdataset import processar_modelos_subset , plotar_graficos, gerar_tabelas_percentuais_metricas
df = pd.read_csv("resultados.csv")
processar_modelos_subset(df,roc_curve=True,
                              f1_score=True,
                              gmean=True,
                              confusion_matrix=True,
                              prec_recall_curve=True,
                              modo_visual='juntos')
plotar_graficos("resultados.csv",roc_vs_ratio=True,
                                prec_rec_vs_ratio=True,
                                gmean_vs_ratio=True,
                                modo_ratio='weighted',
                                modo_visual='todos')
gerar_tabelas_percentuais_metricas(df, limiares=(0 ,0.2,0.5, 0.7, 0.8, 0.9),min_test_size=200)

In [ ]:
from teste_hipoteses import comparar_losses_metricas
df = pd.read_csv("resultados.csv")
comparar_losses_metricas(df, alpha=0.05)